# Challenge - GOT Snapchat Filter?
## GOT Snapchat Filter?
Making cool face filters for GOT Characters & yourself!
In this problem, you will be building snapchat like face filters - eyeglasses and moustache for popular Game Of Throne Characters Tyrion Lannister and Jaime.

<table>
    <tr>
    <td>
        <img src="136339.svg" alt="drawing" width="120"/> 
    </td>
    <td>
        <img src="136346.svg" alt="drawing" width="120"/> 
    </td>
    </tr>
</table>



 

- You will be given an input image and two image templates(one for sunglasses & moustache , you task is to overlay the eyeglasses and moustache on the given character. A sample input output is show below. To detect the facial keypoints like position of eyes and nose, we also provide you with haarcascade xml files in the training data. 

<img src = 'jamie.jpg' width = "200"/>

### Bonus -
Try to place these filters in LIVE stream taken from your webcam and display it :)

### Submission Format
For the given input image, submit a CSV file where each ith row contains the BGR pixel values for the ith pixel of modified image.

### Scoring
Total score will be based upon percentage of pixel values correctly predicted. You should flatten the image into shape(-1,3) before creating CSV.

In [1]:
import pandas as pd
import numpy as np
import cv2

In [2]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]
    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image
    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)
    # return the resized image
    return resized

In [3]:
image = cv2.imread('Data/Test/Before.png')
glasses = cv2.imread('Data/Train/glasses.png', -1)
mustache = cv2.imread('Data/Train/mustache.png', -1)
eye_cascade = cv2.CascadeClassifier('frontalEyes35x16.xml')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
nose_cascade = cv2.CascadeClassifier('Data/Train/third-party/Nose18x15.xml')

In [4]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

In [5]:
# iterating over the face detected
for (x, y, w, h) in faces:

    # create two Regions of Interest.
    #cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,0),2) 
    roi_gray = gray[y:y + h, x:x + h]
    roi_color = image[y:y + h, x:x + h]
    eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)
    for (ex, ey, ew, eh) in eyes:
        #cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,127,255),2) 
        roi_eyes = roi_gray[ey: ey+eh, ex: ex+ew]
        glasses2 = image_resize(glasses.copy(), width = ew)
        gw, gh, gc = glasses2.shape
        
        for i in range(0, gw):
        	for j in range(0, gh):
        		if(glasses2[i, j][3] != 0):
        			roi_color[ey + i, ex + j] = glasses2[i, j]

    
    nose = nose_cascade.detectMultiScale(roi_gray, 1.3, 5)
    for (nx, ny, nw, nh) in nose:
        #cv2.rectangle(roi_color,(nx,ny),(nx+nw,ny+nh),(120,127,255),2)
        roi_nose = roi_gray[nx: ny+nh, nx: nx+nw]
        mustache2 = image_resize(mustache.copy(), width = nw)

        mw, mh, mc = mustache2.shape
        for i in range(0, mw):
        	for j in range(0, mh):
        		if( mustache2[i, j][3] != 0):
        			roi_color[ny + int(nh/2.0) + i, nx+j] = mustache2[i, j]

        #print(nose) 
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    cv2.imshow('image', image)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [15]:
image1 = image.reshape((-1, 3))
print(image1.shape)

(182845, 3)


In [24]:
df = pd.DataFrame(image1, columns=["Channel 1","Channel 2","Channel 3"])

In [25]:
df.to_csv('output.csv', index = False)